## Install Prerequisites

In [1]:
import subprocess
import sys
import base64
import getpass
import requests
import yaml
import matplotlib
import plotly.graph_objects as go
import networkx as nx
import webbrowser

# Client Configuration

In [71]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import TwoByTwoLayout
from ipywidgets import AppLayout, Button, Layout


sdia_conf_url_widget =  widgets.Text(
    value='https://lifewatch.lab.uvalight.net:30736/orchestrator',
    placeholder='Enter SDIA API endpoint. e.g. https://lifewatch.lab.uvalight.net:30736/orchestrator',
    description='SDIA API URL:',
    disabled=False
)
sdia_conf_username_widget =  widgets.Text(
    value='notebook_user',
    placeholder='Enter SDIA username',
    description='SDIA username:',
    disabled=False
)

sdia_conf_token_widget =  widgets.Password(
    value='',
    placeholder='Enter your SDIA token',
    description='Password:',
    disabled=False
) 

sdia_conf_login_button = widgets.Button(
    description='Login',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run report',
#     icon='check' # (FontAwesome names without the `fa-` prefix)
)

sdia_conf_output = widgets.Output()
sdia_conf_box = widgets.VBox([sdia_conf_url_widget, sdia_conf_username_widget, sdia_conf_token_widget, sdia_conf_login_button])
display(sdia_conf_box,sdia_conf_output)

@output.capture()
def sdia_conf_on_button_clicked(b):
    sdia_conf_url = sdia_conf_url_widget.value
#     if not sdia_token_widget.value or not sdia_username_widget.value:
        

sdia_conf_login_button.on_click(sdia_conf_on_button_clicked)





Output()

# Application Configuration

In [96]:
application_type_widget = widgets.Dropdown(
    options=['Docker', 'Helm'],
    value='Helm',
    description='Application Type:',
    disabled=False
)

application_type_output = widgets.Output()
application_type_box = widgets.VBox([application_type_widget])
display(application_type_box,application_type_output)

Output()

In [97]:
docker_app_image_name_widget =  widgets.Textarea(
    value='cloudcells/classifiers',
    placeholder='image name e.g. cloudcells/classifiers',
    description='Docker Image Name:',
    disabled=False
)

docker_app_image_ports_widget =  widgets.Textarea(
    value='',
    placeholder='the ports to expose e.g. 80',
    description='Ports:',
    disabled=False
)

enebale_monotoring_widget= widgets.Checkbox(
    value=True,
    description='Monotoring',
    disabled=False
)

docker_app_box = widgets.VBox([docker_app_image_name_widget, docker_app_image_ports_widget,enebale_monotoring_widget])

                    
helm_app_chart_name_widget =  widgets.Text(
    value='argo/argo',
    placeholder='the chart name e.g. argo/argo',
    description='Chart Name:',
    disabled=False
)


helm_app_repo_name_widget =  widgets.Text(
    value='argo',
    placeholder='the name of the repoitory name',
    description='Reposetory Name:',
    disabled=False
)

helm_app_repo_url_widget =  widgets.Text(
    value='https://argoproj.github.io/argo-helm',
    placeholder='the url',
    description='Reposetory URL:',
    disabled=False
)

helm_app_box = widgets.VBox([helm_app_chart_name_widget, helm_app_repo_name_widget,helm_app_repo_url_widget,enebale_monotoring_widget])

app_conf_output = widgets.Output()

if application_type_widget.value == 'Docker':
    display(docker_app_box,docker_app_output)
elif application_type_widget.value == 'Helm':
    display(helm_app_box,docker_app_output)


Output()

# Infrastructure Configuration

## Number of Topologies
A topology is a set of VMs that will be provisined in the same location by the same provider.

In [77]:
num_of_topologies_widget= widgets.BoundedIntText(
    value=1,
    min=1,
    max=4,
    step=1,
    description='Num. Of Topologies:',
    disabled=False
)


topologies_output = widgets.Output()
number_of_topology_box = widgets.VBox([num_of_topologies_widget])
display(number_of_topology_box,topologies_output)


Output()

In [142]:
topologies_boxes = []
for i in range(0, num_of_topologies_widget.value):
    cloud_provider_name_widget = widgets.Dropdown(
        options=['EC2', 'Azure'],
        value='Azure',
        description='Cloud Provider:',
        name='cloud_provider_name',infrastructure
        disabled=False
    )
    
    num_of_vms_widget= widgets.BoundedIntText(
        value=2,
        min=1,
        max=4,
        step=1,
        description='Num. Of VMs:',
        disabled=False
    )
    vm_size_name_widget = widgets.Dropdown(
        options=['small', 'medium','large'],
        value='medium',
        description='VM size:',
        disabled=False
    )
    topology_region_widget = widgets.Dropdown(
        options=['East US','Central US','South Central US','West US','Australia East','Southeast Asia','UK South','West Europe','North Europe','South Africa North','Central India'],
        value='West Europe',
        description='Topology Region:',
        disabled=False
    )
    topology_box = widgets.VBox([cloud_provider_name_widget, num_of_vms_widget, vm_size_name_widget,topology_region_widget])
    topologies_boxes.append(topology_box)

    
    
topologies_conf_tab = widgets.Tab()
topologies_conf_tab.children = children
for i in range(len(topologies_boxes)):
    topologies_conf_tab.set_title(i, 'topology_'+str(i+1))
    
    
topologies_conf_output = widgets.Output()
topologies_conf_tab_box = widgets.VBox([topologies_conf_tab])
display(topologies_conf_tab_box,topologies_conf_output)

Output()

## Generate TOSCA

In [156]:
node_templates={}
app_prop = []
if application_type_widget.value == 'Docker':
    app_prop.append('Image Name: '+docker_app_image_name_widget.value)
    app_prop.append('Ports: '+docker_app_image_ports_widget.value)
elif application_type_widget.value == 'Helm':
    app_prop.append('Application Type: '+application_type_widget.value)
    app_prop.append('Chart name: '+helm_app_chart_name_widget.value)
    app_prop.append('Reposetory name: '+helm_app_repo_name_widget.value)
    app_prop.append('Reposetory URL: '+helm_app_repo_url_widget.value)

text = 'Application\n'
for line in app_prop:
    text += line+'\n'
    
    
text += '\nInfrastructure\n'


for topology in topologies_boxes:
    for child in topology.children:
        text+=child.description+':'+str(child.value)+'\n'
    
    text+='-----\n'

print(text)



topology_template['node_templates'] = node_templates
tosca = {}
tosca['tosca_definitions_version'] = 'tosca_simple_yaml_1_2'
imports=[{'nodes':'https://raw.githubusercontent.com/qcdis-sdia/sdia-tosca/master/types/nodes.yaml','data':'https://raw.githubusercontent.com/qcdis-sdia/sdia-tosca/master/types/data.yml','capabilities':'https://raw.githubusercontent.com/qcdis-sdia/sdia-tosca/master/types/capabilities.yaml','policies':'https://raw.githubusercontent.com/qcdis-sdia/sdia-tosca/master/types/policies.yaml','interfaces':'https://raw.githubusercontent.com/qcdis-sdia/sdia-tosca/master/types/interfaces.yml'}]
tosca['imports'] = imports
repositories = {'docker_hub':'https://hub.docker.com/'}
tosca['repositories'] = repositories
tosca['topology_template'] = topology_template


Application
Application Type: Helm
Chart name: argo/argo
Reposetory name: argo
Reposetory URL: https://argoproj.github.io/argo-helm

Infrastructure
Cloud Provider::Azure
Num. Of VMs::2
VM size::medium
Topology Region::West Europe
-----
Cloud Provider::Azure
Num. Of VMs::2
VM size::medium
Topology Region::West Europe
-----



NameError: name 'node_templates' is not defined

## Show Topology

In [ ]:



def build_graph(node_templates):
    graph = nx.DiGraph()
    for node_name in node_templates:
        graph.add_node(node_name, attr_dict=node_templates[node_name])
        if 'requirements' in node_templates[node_name]:
            for req in node_templates[node_name]['requirements']:
                req_name = next(iter(req))
                req_node_name = req[req_name]['node']
                if 'relationship' in req[req_name] and 'type' in req[req_name]['relationship']:
                    relationship_type = req[req_name]['relationship']['type']
                else:
                    if 'relationship' not in req[req_name]:
                        relationship_type = 'tosca.relationships.DependsOn'
                    else:
                        relationship_type = req[req_name]['relationship']
                graph.add_edge(node_name, req_node_name, relationship=relationship_type)

    # nx.draw(graph, with_labels=True)
    # plt.savefig("/tmp/graph.png")
    # plt.show()
    return graph

def get_tosca(tosca_id):
    path = "/tosca_template/"+tosca_id
    payload = {}
    headers= {'accept':'text/plain'}
    response = requests.request("GET", base_url+path, headers=headers, data = payload,verify=False, auth=(username, password))
    return response.text

tosca = get_tosca(tosca_id)
tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)


## Make Plan

In [ ]:
path = "/planner/plan/"+tosca_id
payload = {}
headers= {'accept':'text/plain'}

response = requests.request("GET", base_url+path, headers=headers, data = payload,verify=False, auth=(username, password))
planed_tosca_id = response.text

In [ ]:
tosca = get_tosca(planed_tosca_id)

tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)

for node_name in tosca_dict['topology_template']['node_templates']:
    if 'properties' in tosca_dict['topology_template']['node_templates'][node_name]:
        print(node_name+': '+str(tosca_dict['topology_template']['node_templates'][node_name]['properties']))
        print('---------------------------------------------------------------------------------------------------------------------')
        
        

## Provision Plan

In [ ]:
path = "/provisioner/provision/"+planed_tosca_id
payload = {}
headers= {'accept':'text/plain'}

response = requests.request("GET", base_url+path, headers=headers, data = payload,verify=False, auth=(username, password))
provisioned_tosca_id = response.text

In [ ]:
tosca = get_tosca(provisioned_tosca_id)

tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)


for node_name in tosca_dict['topology_template']['node_templates']:
    if tosca_dict['topology_template']['node_templates'][node_name]['type'] == 'tosca.nodes.QC.VM.Compute':
        print(node_name+': '+tosca_dict['topology_template']['node_templates'][node_name]['attributes']['public_ip'])

## Deploy K8s And WS

In [ ]:
path = "/deployer/deploy/"+provisioned_tosca_id
payload = {}
headers= {'accept':'text/plain'}

response = requests.request("GET", base_url+path, headers=headers, data = payload, verify=False, auth=(username, password))
deployed_tosca_id = response.text

In [ ]:
tosca = get_tosca(deployed_tosca_id)

tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)

for node_name in tosca_dict['topology_template']['node_templates']:
    if tosca_dict['topology_template']['node_templates'][node_name]['type'] == 'tosca.nodes.QC.docker.Orchestrator.Kubernetes':
        for token in tosca_dict['topology_template']['node_templates'][node_name]['attributes']['tokens']:
            if token['user'] == 'dashboard_token':
                print(node_name+': '+' dashboard_token: '+token['token'])
        print(node_name+': '+str(tosca_dict['topology_template']['node_templates'][node_name]['attributes']['dashboard_url']))
    if tosca_dict['topology_template']['node_templates'][node_name]['type'] == 'tosca.nodes.QC.Container.Application.Docker':
        print(node_name+': '+str(tosca_dict['topology_template']['node_templates'][node_name]['attributes']['service_url']))
        